In [159]:
import sqlite3
import pandas as pd
import json
import numpy as np
import requests
import time
import datetime as DT
import sys
import threading
import os

pd.set_option('display.max_rows', None)

# Create the connection
con = sqlite3.connect(r'niftyOptionChainAnalysis.db')
cur = con.cursor()
symbol = 'NIFTY'
symbols=['NIFTY', 'BANKNIFTY']

tableprefix = "optionChainWithVolume_"
default_table = tableprefix+symbol
#runatStart(symbol)
strike_range = 5; ## in % to be selected
firstPID = 0;
latestData = {}; # this will store the latest option chain data as a mapping of symbol
#next weekl
nearWeekExpiry = "28-May-2020";
nearMonthExpirDate="28-May-2020";
nextMonthExpiryDate = "25-Jun-2020";
syncTimeDelay = 4; ## in minutes time after which data will be fetched from NSE

#totalTradedVolume,totalBuyQuantity,totalSellQuantity -- add this also for full analysis
columnNames = "strikePrice, expiryDate, openInterest, changeinOpenInterest,impliedVolatility," \
              " lastPrice, change, types,internalValue, externalValue,underlyingPrice,timestamp," \
              "totalTradedVolume,totalBuyQuantity,totalSellQuantity"
columnNames_list = ['strikePrice', 'expiryDate', 'openInterest', 'changeinOpenInterest',
       'impliedVolatility', 'lastPrice', 'change', 'types', 'internalValue',
       'externalValue', 'underlyingPrice', 'timestamp','totalTradedVolume','totalBuyQuantity', 'totalSellQuantity'] #'totalTradedVolume','totalBuyQuantity', 'totalSellQuantity'

# cur.execute('CREATE TABLE optionChain_'+symbol+' (strikePrice, expiryDate, openInterest, changeinOpenInterest,impliedVolatility, lastPrice, change, types, underlyingPrice,timestamp, internalValue, externalValue)')
def onetimeSetup(symbol):
    cur.execute(
        'CREATE TABLE' + tableprefix + symbol + '('+columnNames+')')

def runatStart(symbol):
    cur.execute('SELECT * FROM '  + tableprefix + symbol + ';')


# this function execute a given query and return the result in dataframe format

# query could be like 'SELECT * FROM optionChain_nifty'
def executeSQLQuery(query):
    query = query.replace('*',columnNames)
    print("Executing Query : "+ query);
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns = columnNames_list)
    # for row in cur.execute('SELECT * FROM optionChain_nifty;'):
    #     print(row)
    return df;


# this function is just an helper function
# it calls th NSE website and gets the current option-chain dat for a given symbol
def getOptionChainDataFromNSEfor(symbol):
    url ="https://www.nseindia.com/api/option-chain-indices?symbol="+symbol
    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    # Pull NSE option chain
    r = requests.get(url, headers=header)
    if (r.status_code != 200):
        print("some error Occured in fetching data from NSE website Status: " + r.status_code);
        return;
    return r.content;


# Convert html page as Table and read the first table which has option data
#this function fetch data from nse website and process them to a dataframe,
# it also does useful calculation like internal value and date addition.
def getProcessedOptionChainData(symbol):
    # with open('response.json') as json_file:
    #     data = json.load(json_file)
    #     allRecord = data['records']['data'];
    data = json.loads(getOptionChainDataFromNSEfor(symbol))
    allRecord = data['records']['data'];
    allDates = data['records']['expiryDates'];
    nearestExpiryDate = allDates[0];
    flattenedRecord = []
    underlyingPrice = data['records']['underlyingValue'];
    timeSync = data['records']['timestamp']; ## time when the data was synced with the server
    for oneRecord in allRecord :
        if('CE' in oneRecord):
            oneOptionRecord = oneRecord['CE']
            oneOptionRecord['types'] = 'CE'
            flattenedRecord.append(oneOptionRecord)
            #underlyingPrice = oneOptionRecord['underlyingValue']
            oneOptionRecord['internalValue'] = np.absolute(oneOptionRecord['strikePrice'] - oneOptionRecord['underlyingValue'])
            oneOptionRecord['externalValue'] = oneOptionRecord['lastPrice'];
            if(oneOptionRecord['strikePrice'] < oneOptionRecord['underlyingValue']):
                oneOptionRecord['externalValue'] = oneOptionRecord['lastPrice']-oneOptionRecord['internalValue']
            else :
                oneOptionRecord['internalValue'] = 0;
            if (oneOptionRecord['externalValue'] < 0):
                oneOptionRecord['externalValue'] = 0

        if('PE' in oneRecord):
            oneOptionRecord = oneRecord['PE']
            oneOptionRecord['types'] = 'PE'
            flattenedRecord.append(oneOptionRecord)
            #underlyingPrice = oneOptionRecord['underlyingValue']
            oneOptionRecord['internalValue'] = np.absolute(oneOptionRecord['strikePrice'] - oneOptionRecord['underlyingValue'])
            oneOptionRecord['externalValue'] = oneOptionRecord['lastPrice'];
            if(oneOptionRecord['strikePrice'] > oneOptionRecord['underlyingValue']):
                oneOptionRecord['externalValue'] = oneOptionRecord['lastPrice']-oneOptionRecord['internalValue'];
            else :
                oneOptionRecord['internalValue'] = 0;
            if (oneOptionRecord['externalValue'] < 0):
                oneOptionRecord['externalValue'] = 0
    lower_Range = np.floor((1-strike_range/100)*underlyingPrice)
    upper_Range = np.floor((1+strike_range/100)*underlyingPrice)

    normalize_data = pd.json_normalize(flattenedRecord)
    option_data = pd.DataFrame.from_dict(normalize_data)
    option_data = option_data.drop(columns=['identifier','underlyingValue','underlying','pchangeinOpenInterest',
                                            'pChange','bidQty','bidprice','askQty','askPrice']);
    FilteredOptionData = option_data[option_data['strikePrice']> lower_Range]
    FilteredOptionData = FilteredOptionData[option_data['strikePrice']<upper_Range]
    FilteredOptionData = FilteredOptionData[(option_data['expiryDate'] == nearWeekExpiry) | (option_data['expiryDate'] == nearMonthExpirDate)
                             | (option_data['expiryDate'] == nextMonthExpiryDate)]
    FilteredOptionData['underlyingPrice'] = underlyingPrice
    FilteredOptionData['timestamp'] = timeSync; #DT.datetime.now().strftime("%m/%d/%Y %H:%M:%S"); #timeSync
    FilteredOptionData = FilteredOptionData[columnNames_list] # reorder in a given order of columns
    return FilteredOptionData.round(2);

    #FilteredOptionData['internalValue'] = np.absolute(FilteredOptionData['strikePrice'] - underlyingPrice)
    # FilteredOptionData['externalValue'] = FilteredOptionData['lastPrice'];
    # df = FilteredOptionData
    # for index, row in df.iterrows():
    #     if(row['types']=='CE'):
    #         if(row['strikePrice']<underlyingPrice):
    #             row['externalValue']= row['lastPrice'] - row['internalValue']
    #     if(row['types']=='PE'):
    #         if(row['strikePrice']>underlyingPrice):
    #             row['externalValue']= row['lastPrice'] - row['internalValue']
    # FilteredOptionData['types']
    # FilteredOptionData

In [105]:
from bokeh.io import output_file, show
from bokeh.models import RangeSlider,Select
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, DateRangeSlider, CheckboxButtonGroup,RadioButtonGroup

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
output_notebook()
output_file("range_slider.html")


Loading BokehJS ...

In [174]:
expiryDates = [nearWeekExpiry,nearMonthExpirDate,nextMonthExpiryDate, "All Above"]
allSrc = {}



    #return ColumnDataSource(callPutMerged_df);
#make_dataset(9000,'28-May-2020', 'NIFTY')
#df = getProcessedOptionChainData("NIFTY");
#latestData[symbol] = df


In [192]:
    
# symbols = ['NIFTY', 'BankNIFTY']
# allTabs = []
# controls = WidgetBox()
# for symbol in symbols:
#     layout = column(expiry_date_selector)
#     tab = Panel(child=layout, title = symbol)
#     allTabs.append(tab)

# tabs = Tabs(tabs=allTabs)

# show(tabs)


Executing Query : SELECT strikePrice, expiryDate, openInterest, changeinOpenInterest,impliedVolatility, lastPrice, change, types,internalValue, externalValue,underlyingPrice,timestamp,totalTradedVolume,totalBuyQuantity,totalSellQuantity FROM optionChainWithVolume_NIFTY WHERE strikePrice in (9300) AND expiryDate='28-May-2020'
CE
PE


In [201]:
def modify_doc(doc):
    
    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '10pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '4pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '4pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '6pt'
        p.yaxis.major_label_text_font_size = '6pt'

        return p
    def make_plot(src,strikePrice):
            # Blank plot with correct labels
            # color=["firebrick", "navy"]
            # Hover tool with vline mode
            hover = HoverTool(tooltips=[
                                        ('Int. Val. call', '@internalValue_x'), 
                                        ('Int. Val. put', '@internalValue_y'), 
                                        ('Underlying Price', '@underlyingPrice')
    #                                     ('No. Of Days', '@frequency'),
    #                                    ('Fraction', '@proportion'),
    #                                    ('pdf', '@pdf')
                                    ],
                              mode='vline')
            
            
            p = figure(plot_width = 400, plot_height = 400, 
                      title = 'OI call : '+strikePrice,x_axis_type='datetime',
                      x_axis_label = 'Time', y_axis_label = 'OI(in # of contract)')

            p.line('timestamp',  'CE_OI', source = src,color = "firebrick", line_width=4, alpha=0.7, legend_label="Call OI")
            p.circle('timestamp',  'CE_OI', source = src,fill_color="white", size=8)

            p.add_tools(hover)
            p = style(p)
            p1 = p
            p = figure(plot_width = 400, plot_height = 400, 
                      title = 'OI Put :' +strikePrice ,x_axis_type='datetime',
                      x_axis_label = 'Time', y_axis_label = 'OI(in # of Contracts)')
            p.line('timestamp',  'PE_OI', source = src,color = "navy", line_width=4, alpha=0.7, legend_label="Put OI")
            p.circle('timestamp',  'PE_OI', source = src,fill_color="white", size=8)

            p.add_tools(hover)
            p = style(p)
            p2 = p
            return row(p1,p2)
    
    def make_dataset(strike_prices, expiry_date,symbol):
        if(type(strike_prices) ==int):
            strike_prices = "(" + str(strike_prices)+")" ## in case tuple length is one it treats it as one single variable
        rawQuery = "SELECT * FROM {tableName} WHERE strikePrice in {strikePrices} AND expiryDate='{expiryDate}'"
        # remove the expiry date clause if all have to be shown
        if(expiry_date =="All Above" ):
            rawQuery = rawQuery.split(' AND expiryDate')[0];
        query = rawQuery.format(tableName=tableprefix + symbol, strikePrices=str(strike_prices),expiryDate=expiry_date);
        df = executeSQLQuery(query);
        df = df.drop(columns=['impliedVolatility','expiryDate','changeinOpenInterest','change',
                              'totalTradedVolume','totalBuyQuantity','totalSellQuantity'])
        df['timestamp'] = df['timestamp'].apply(pd.to_datetime)
        ## have to work on this logic
        if(expiry_date =="All Above"):
            df = df.groupby(['timestamp', 'strikePrice','types'],as_index=False)['openInterest'].sum();
        df = df.drop_duplicates(subset=['strikePrice','timestamp','types'], keep='last');  # removing the duplicate entries        
        #assuming only one strike price is passed at a time as of now
        callPut_df = []
        grouped = df.groupby('types');
        for name,group in grouped:
            print(name);
            #print(group)
            group = group.rename(columns={"openInterest": name+"_OI"}) #"changeinOpenInterest" :name+"_OI change"
            callPut_df.append(group)
        callPutMerged_df = pd.merge(callPut_df[0],callPut_df[1],
                                    left_on=['strikePrice','underlyingPrice','timestamp'],
                                    right_on=['strikePrice','underlyingPrice','timestamp']) #assuming strike_price is same, expiry_date is same
        callPutMerged_df = callPutMerged_df.sort_values(['timestamp'])
        return ColumnDataSource(callPutMerged_df);
    
    def update(attr, old, new):
        selected_expiryDate = expiry_date_selector.value;
        selectedStrikePrice = allUniqueStrikePrice[strikePrice_selector.active];
        new_src = make_dataset(int(selectedStrikePrice),selected_expiryDate,symbol)
        src.data.update(new_src.data)

    allUniqueStrikePrice =latestData[symbol]['strikePrice'].apply(str).unique().tolist();
    ATMStrikeindex = int(np.floor(len(allUniqueStrikePrice)/2))
    expiry_date_selector = Select(title="Select Expiry Date", value=nearWeekExpiry, options=expiryDates)
    #strikePrice_selection = CheckboxButtonGroup(labels=available_days, active=[ATMStrikeindex-1, ATMStrikeindex,ATMStrikeindex+1]) # in case multiple to be shown at once#
    strikePrice_selector = RadioButtonGroup(
            labels=allUniqueStrikePrice, active=ATMStrikeindex);


    selected_expiryDate = expiry_date_selector.value;
    selectedStrikePrice = allUniqueStrikePrice[strikePrice_selector.active];


    src = make_dataset(int(selectedStrikePrice),selected_expiryDate,symbol)
    p = make_plot(src,selectedStrikePrice)


    strikePrice_selector.on_change('active', update)
    expiry_date_selector.on_change('value', update)

    layout = column(row(expiry_date_selector,strikePrice_selector),p)
    doc.add_root(layout)

    
handler = FunctionHandler(modify_doc)
app = Application(handler)


In [202]:
show(app)

Executing Query : SELECT strikePrice, expiryDate, openInterest, changeinOpenInterest,impliedVolatility, lastPrice, change, types,internalValue, externalValue,underlyingPrice,timestamp,totalTradedVolume,totalBuyQuantity,totalSellQuantity FROM optionChainWithVolume_NIFTY WHERE strikePrice in (9300) AND expiryDate='28-May-2020'
CE
PE
Executing Query : SELECT strikePrice, expiryDate, openInterest, changeinOpenInterest,impliedVolatility, lastPrice, change, types,internalValue, externalValue,underlyingPrice,timestamp,totalTradedVolume,totalBuyQuantity,totalSellQuantity FROM optionChainWithVolume_NIFTY WHERE strikePrice in (9150) AND expiryDate='28-May-2020'
CE
PE


In [210]:
from numpy import arange, linspace, pi, sin

from bokeh.models import LinearAxis, Range1d
from bokeh.plotting import figure,  show

x = arange(-2*pi, 2*pi, 0.1)
y = sin(x)
y2 = linspace(0, 100, len(y))

p = figure(y_range=(-1.1, 1.1))

p.circle(x, y, color="blue")

p.extra_y_ranges = {"foo": Range1d(start=0, end=100)}
p.circle(x, y2, color="blue", y_range_name="foo")

show(p)